# Automatic Sustainability Objective Detection

## === Setup ===

### Importing Libraries

In [11]:
import sys
import pandas
import IPython.display

sys.path.append("../../goalspotter_core/source")
import document
import data_preprocessing
import transformer_model

pandas.set_option("display.max_rows", None)
pandas.set_option("display.max_columns", None)
pandas.set_option("display.max_colwidth", None)

### Setting up the Data Preprocessor

In [12]:
data_preprocessor = data_preprocessing.DataPreprocessing()

### Loading Our Trained Models

In [13]:
target_values = ["Not Goal", "Goal"]
goal_detection_model = transformer_model.TextClassification(target_values, name="distilroberta-base", load_from="../../goalspotter_core/models/goal-detection")

### Objective Extraction Helper Function

In [14]:
def extract_objectives_from_url(url, content_type="pdf"):
    
    # Extracting Text Blocks of the Sustainability Report
    doc = document.Document(url)
    doc.content_type = content_type
    content = doc.read_local_file()
    parsed_content = doc.parse_content(content)
    text_blocks = doc.segment_text(parsed_content)
    sentences = doc.get_sentences(text_blocks)
    tdf = pandas.DataFrame({"Text Blocks": sentences})    
    
    # Running the Goal Detection Model
    tdf["text"] = tdf["Text Blocks"].copy()
    tdf = data_preprocessor.clean_text_blocks(tdf, "text", level="essential")
    tdf = data_preprocessor.filter_text_blocks(tdf, "text", keep_only_size=(0, 300))
    predictions = goal_detection_model.predict(tdf["text"].tolist())
    tdf["Goal Score"] = predictions["Goal"].values
    tdf = tdf.drop(["text"], axis=1)
    tdf = tdf.sort_values("Goal Score", ascending=False)    
    
    tdf = tdf.fillna("")
    return tdf    

## === Processing New Sustainability Reports ===

In [ ]:
file_name = "google_2021"
url = f"../documents/google/{file_name}.pdf"
# IPython.display.display(IPython.display.IFrame(f"file://{url}", width=1000, height=800))
df = extract_objectives_from_url(url)
df = df.drop_duplicates(subset="Text Blocks", keep="first")
df = df[df["Text Blocks"].str.len() >= 25]
df = df[df["Goal Score"] >= 0.1]
df.insert(0, "ID", range(1, len(df) + 1))
df.to_csv(f"../objectives/google/{file_name}.csv", index=False)
df

ID  \
271   1   
278   2   
362   3   
90    4   
342   5   
343   6   
276   7   
357   8   
280   9   
328  10   
226  11   
305  12   
266  13   
234  14   
301  15   
60   16   
134  17   
364  18   
296  19   
353  20   
308  21   
294  22   
321  23   
371  24   
324  25   
312  26   
87   27   
355  28   
879  29   
373  30   
101  31   
375  32   
135  33   
249  34   
161  35   

                                                                                                                                                                                                          Text Blocks  \
271                                                                                Maintain ISO 50001 energy management system certification for Google-owned data centers  that meet certain operational milestones.   
278                                                                                                                                   In 2020, our global landfill diversion rate for data center operations was 81%.   
362                                                                                        Achieve UL 2799 Zero Waste to Landfill certification  at all final assembly consumer hardware manufacturing sites by 2022.   
90                                                                                                                                         replenish more water than we consume by 2030 and to support water security   
342                                                                                        Achieve carbon neutrality for 100% of shipments  of Made by Google products to and from Google’s direct customers by 2020.   
343                                                                                             Publish product environmental reports for 100%  of flagship consumer hardware products launching  in 2020 and beyond.   
276                                                                                                                                             Achieve Zero Waste to Landfill for our global data center operations.   
357                                                                                                             Include recycled materials in 100% of Made by Google products launching in 2022 and every year after.   
280                                                                                                                         Replenish 120% of the water we consume, on  average, across our offices and data centers.   
328                        Pursue the ILFI Living Building Challenge Certification for our Charleston East and Bay View campuses— two of Google’s first ground-up development  projects at our Bay Area headquarters.   
226  In 2020, we expanded our take-back program to all 32 countries where we ship Made by Google products, allowing customers to responsibly recycle old and unused devices  for free—whether made by  Google or not.   
305                                                                                                                                 Achieve net-zero emissions across all of our  operations and value chain by 2030.   
266                                                                                                                     Maintain or improve average annual fleet-wide  PUE across Google data centers year over year.   
234                                                                                                                          By the end of 2020, Google Maps contained nearly 180,000 EV charging locations globally.   
301                                                                                                                                  Enable 5 GW of new carbon-free energy in our  key manufacturing regions by 2030.   
60                                                                                                                                      